<a href="https://colab.research.google.com/github/chrismoroney/job-search-using-llm/blob/main/Integrated_job_search_with_docs_sheets_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Import Packages</h1>

In [1]:
!pip install openai pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 33.3 MB/s eta 0:00:00


<h1>Upload Resume</h1>

In [2]:
from google.colab import files
import fitz  # pymupdf

print("📄 Upload your resume (PDF format)")
uploaded_resume = files.upload()
resume_path = list(uploaded_resume.keys())[0]

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

resume_text = extract_text_from_pdf(resume_path)
print("Resume text extracted successfully!")

📄 Upload your resume (PDF format)


Saving Moroney updated Resume 202503.pdf to Moroney updated Resume 202503.pdf
Resume text extracted successfully!


In [18]:
from openai import OpenAI
from google.colab import userdata

# Replace with own API key
api_key = userdata.get('OPEN_AI_KEY')
client = OpenAI(api_key=api_key)

In [19]:
import requests

def search_jobs(title="software engineer", location="seattle"):
    url = "https://jsearch.p.rapidapi.com/search"

    querystring = {
        "query": f"{title} in {location}",
        "page": "1",           # pagination available
        "num_pages": "1"
    }

    # Replace with own API key
    headers = {
        "x-rapidapi-key": userdata.get('X-RAPIDDAPI_KEY'),
        "x-rapidapi-host": "jsearch.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        return response.json().get("data", [])
    else:
        print("Error:", response.status_code)
        return []

In [20]:
import re

def extract_score(response_text):
    match = re.search(r"(\d{1,3})", response_text)
    return int(match.group(1)) if match else 0

def match_jobs(resume, jobs):
    results = []
    for job in jobs:
        job_desc = job.get("job_description", "")
        prompt = f"""Given the resume:
---
{resume}

Evaluate its fit for the job below and rate from 0–100.
Put a heavy emphasis on professional job experience required for the job and the resume provides.
For instance if a job requires 5+ years of experience, score significantly less overall if the resume does not provide enough experience.
Additionally, put a small emphasis in the score on the user's own interests and skills either in the summary or job experience.
Explain briefly:
---
{job_desc}
"""
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        content = response.choices[0].message.content
        score = extract_score(content)
        results.append({**job, "score": score, "reason": content})
    return sorted(results, key=lambda x: x['score'], reverse=True)

def generate_cover_letter(resume, job_description):
    prompt = f"""Write a short, enthusiastic, tailored cover letter using the resume below for the following job.
Resume:
---
{resume}

Job Description:
---
{job_description}
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


In [21]:
cover_letters_dict = {}  # Initialize dictionary to store cover letters
# Customize your job search here
search_title = "software engineer"
search_location = "san francisco"

print(f"🔍 Searching for: {search_title} in {search_location}...")
job_listings = search_jobs(search_title, search_location)
print(f"✅ Found {len(job_listings)} jobs")
top_matches = match_jobs(resume_text, job_listings[:5])  # adjust number as needed

for job in top_matches:
    print(f"\n📌 Job: {job['job_title']} at {job['employer_name']}")
    print(f"🔗 Location: {job['job_apply_link']}")
    print(f"📍 Location: {job['job_city']}, {job['job_country']}")
    print(f"🔢 Match Score: {job['score']}")
    print(f"🗣 Reason: {job['reason']}")
    print("\n📝 Cover Letter:\n")
    cover_letter = generate_cover_letter(resume_text, job['job_description'])
    job_id = f"{job['job_title']} at {job['employer_name']}"
    cover_letters_dict[job_id] = cover_letter
    print(cover_letter)
    print("\n" + "="*80 + "\n")

🔍 Searching for: software engineer in san francisco...
✅ Found 10 jobs

📌 Job: Product Software Developer (Full-Stack) at Jobot
🔗 Location: https://www.linkedin.com/jobs/view/product-software-developer-full-stack-at-jobot-4210261679?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic
📍 Location: San Francisco, US
🔢 Match Score: 85
🗣 Reason: Score: 85

The candidate has a strong background in full-stack development and working with AI, addressing the company's requirement for a full-stack developer able to integrate AI tools. He has over three years of professional experience, meeting the minimum experience requirement, and has worked on building and shipping impactful product features.

The candidate has experience with AI/ML, with a certificate from TensorFlow and an internship at Deeplearning.ai. Also, he has used Node.js and React.js, and worked on APIs and data platforms.

However, the candidate's experiences don't exactly map onto all requirements for th

In [22]:
# 📌 Authenticate with Google & set up APIs
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaInMemoryUpload
import datetime

creds, _ = default()
gc = gspread.authorize(creds)
docs_service = build('docs', 'v1', credentials=creds)
drive_service = build('drive', 'v3', credentials=creds)

In [23]:

# 📄 Save cover letter to Google Doc and return shareable link
def save_cover_letter_to_doc(title, content, folder_name="Generated Cover Letters"):
    # 1. Search for the folder (create it if not found)
    folder_id = None
    response = drive_service.files().list(
        q=f"mimeType='application/vnd.google-apps.folder' and name='{folder_name}' and trashed = false",
        spaces='drive',
        fields='files(id, name)',
    ).execute()
    folders = response.get('files', [])
    if folders:
        folder_id = folders[0]['id']
    else:
        folder_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder'
        }
        folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
        folder_id = folder.get('id')

    # 2. Create the doc
    doc = docs_service.documents().create(body={"title": title}).execute()
    doc_id = doc.get("documentId")

    # 3. Move it to the folder
    drive_service.files().update(
        fileId=doc_id,
        addParents=folder_id,
        removeParents='root',
        fields='id, parents'
    ).execute()

    # 4. Write content to doc
    requests = [{"insertText": {"location": {"index": 1}, "text": content}}]
    docs_service.documents().batchUpdate(documentId=doc_id, body={"requests": requests}).execute()

    return f"https://docs.google.com/document/d/{doc_id}/edit"


In [24]:
# 📊 Get or create Google Sheet to track job applications
def get_or_create_sheet(sheet_name="Job Tracker"):
    try:
        sheet = gc.open(sheet_name)
    except:
        sheet = gc.create(sheet_name)
        sheet.share('', perm_type='anyone', role='writer')
        sheet.sheet1.append_row(["Date", "Job Title", "Company", "Location", "Score", "Doc Link"])
    return sheet.sheet1

In [25]:
import datetime
import pytz

# ➕ Add a row to Google Sheet
def log_to_sheet(sheet, job, score, doc_link):
    # Get current time in Pacific Time
    pacific = pytz.timezone('US/Pacific')
    now_pacific = datetime.datetime.now(pacific).strftime("%Y-%m-%d")

    row = [
        now_pacific,
        job['job_title'],
        job['employer_name'],
        f"{job['job_city']}, {job['job_country']}",
        score,
        doc_link
    ]
    sheet.append_row(row)


In [26]:
# ✅ Run this to generate cover letters and log to Google Docs/Sheets
sheet = get_or_create_sheet("Job Tracker")

# Get and sort matched jobs ONCE
sorted_matches = sorted(top_matches, key=lambda x: x['score'], reverse=True)[:5]  # Top 5 consistent picks

for job in top_matches:
    print(f"\n📌 Job: {job['job_title']} at {job['employer_name']}")
    print(f"🔢 Match Score: {job['score']}")

    # Save to Google Doc
    job_id = f"{job['job_title']} at {job['employer_name']}"
    doc_link = save_cover_letter_to_doc(f"{job['job_title']} - {job['employer_name']}", cover_letters_dict[job_id])

    # Log everything to Google Sheet
    log_to_sheet(sheet, job, job['score'], doc_link)

    print(f"✅ Saved cover letter: {doc_link}")
    print("=" * 80)



📌 Job: Product Software Developer (Full-Stack) at Jobot
🔢 Match Score: 85
✅ Saved cover letter: https://docs.google.com/document/d/10ZY4hWdnEjCobwJjFBtV37JcSQEj-P-ogEd0s-P_ejc/edit

📌 Job: Associate, Software Engineer, Core Platform at BlackRock
🔢 Match Score: 80
✅ Saved cover letter: https://docs.google.com/document/d/1s-X2obngbjmhpWgqBgK1-bx6cUSVGW-e2KbRIn3rZ-I/edit

📌 Job: Software Engineer (Frontend) at Greptile
🔢 Match Score: 80
✅ Saved cover letter: https://docs.google.com/document/d/1yJpSApGGyI5_HVeGxkH17P-WcXs9bhUmN69DdYfMnfs/edit

📌 Job: Software Engineer  (EMU team)-1 at Salesforce, Inc.
🔢 Match Score: 60
✅ Saved cover letter: https://docs.google.com/document/d/1pYcCaU8iG_EyMXeWUbRb_EQnXWn0JDctJejWzIcS_aI/edit

📌 Job: Software Engineer III, Infrastructure, Google Cloud at Google
🔢 Match Score: 2
✅ Saved cover letter: https://docs.google.com/document/d/1Lvlbn9c3v4c5yDjoFQ-rmN10B-I2sRjk7obFf8jMbs8/edit
